## 1- ETL (Extract, Transform and Load) ##

Importamos las librerias necesarias.

In [142]:
import pandas as pd
import numpy as np
import ast

Leemos el archivo de películas y procedemos a analizar la estructura obteniendo los primeros $3$ elementos del dataframe.

In [143]:

df_movies = pd.read_csv('_src/Dataset/movies_dataset.csv')
df_movies.head(3)


C:\Users\germa\AppData\Local\Temp\ipykernel_11364\276130124.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('Dataset/movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0



**Realizaremos todas las modificaciones propuestas.**

Eliminaremos las columnas que no serán utilizadas.

In [144]:
df_movies.drop(columns=['video','imdb_id','adult','original_title','poster_path', 'homepage'], inplace=True)

Rellenaremos todos los campos con valores nulos en las columnas _`revenue`_ y _`budget`_. También, cambiaremos el tipo de dato de _`budget`_ a númerico para poder crear un nuevo campo calculado.

In [145]:
df_movies['revenue'] = df_movies['revenue'].apply(lambda x: x if pd.notnull(x) else 0)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce').fillna(0)


Crearemos el campo  _`return`_ dado con las columnas _`revenue`_ y _`budget`_ evitando la división por 0.

In [146]:
df_movies['return'] = df_movies.apply(lambda x: (x['revenue'] / x['budget']) if x['budget'] != 0 else 0, axis=1)

Cambiaremos el tipo de dato en el campo _`release_data`_ a datetime controlando los errores que pudieran surgir y normalizaremos el formato de las fechas.

In [147]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies['release_date'] = df_movies['release_date'].dt.strftime('%Y-%m-%d')

Eliminaremos los registros con valores faltantes en el campo _`release_data`_.

In [148]:
df_movies.dropna(subset=['release_date'], inplace=True)

Crearemos la columna _`release_year`_ dada por el año de lanzamiento de la película.

In [149]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])
df_movies['release_year'] = df_movies['release_date'].dt.year

Normalizaremos los campos que tienen datos anidados en listas y diccionarios, tomando la información útil.

  Nos quedaremos con los nombres de las colecciones, compañias, géneros, lenguajes hablados y países en los que se produjeron las películas.


In [150]:

df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)

df_movies['genres'] = df_movies['genres'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)

df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)

df_movies['production_countries'] = df_movies['production_countries'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)



En los campos que se enumeren items (_`genres`_, _`spoken_languages`_, etc.) adoptaremos un formato con comas para una lectura más simple.

In [151]:
df_movies['production_companies'] = df_movies['production_companies'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df_movies['genres'] = df_movies['genres'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df_movies['production_countries'] = df_movies['production_countries'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

Leeremos el archivo con la información sobre los actores y directores de las películas.

In [152]:
df_credits = pd.read_csv('Dataset/credits.csv')


Desanidaremos el campo _`cast`_ para obtener el nombre de los actores que participaron en determinada película y haremos lo mismo con _`crew`_ pero tomando únicamente aquella persona que trabajo como **'Director'**.

  Se usará nuevamente un formato con comas entre cada nombre de actor o director.

In [153]:
df_credits['cast'] = df_credits['cast'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df_credits['cast'] = df_credits['cast'].apply(lambda x: ', '.join([i['name'] for i in x]) if isinstance(x, list) else np.nan)

df_credits['crew'] = df_credits['crew'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df_credits['crew'] = df_credits['crew'].apply(lambda x: ', '.join([i['name'] for i in x if i['job'] == 'Director']) if isinstance(x, list) else np.nan)

Llevaremos la información de ambos archivos a un solo dataset que se unirá a través de el _`'id'`_ de película.

In [154]:
# Cambiaremos a tipo de dato númerico la columna id para no tener problemas en la operación.
df_movies['id'] = df_movies['id'].astype('int64')

data = df_movies.merge(df_credits, how='left', on='id')

Daremos un formato común a los títulos de las películas.

In [155]:
data['title'] = data['title'].apply(lambda x: str(x).lower().capitalize().strip())

Importamos el Dataframe generado en formato csv para su uso en las próximas instancias del proyecto.

In [156]:
data.to_csv('_src/Dataset/clean_data.csv', index=False)